In [1]:
# US Stocks - VTI, CRSP U.S. Total Market Index: http://www.crsp.com/fact-sheet-archive

In [2]:
from bs4 import BeautifulSoup
import requests
import re
from io import BytesIO
from PyPDF2 import PdfFileReader
import os
import pandas as pd

In [3]:
def get_report_links(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text,features="html.parser")
    foundURLS = []
    for link in soup.find_all('a'):
        if re.search('crsptm', str(link.get('href')), re.IGNORECASE):
            foundURLS.append(link.get('href'))
    return foundURLS
def get_file_names(urls):
    file_names = []
    for url in urls:
        file_name = url.split('/')[-1]
        file_names.append(file_name)
    return file_names
def download_index_reports(urls): 
    for url in urls: 
        file_name = url.split('/')[-1]    
        print("Downloading file:%s"%file_name) 
        r = requests.get(url, stream = True) 
        with open(file_name, 'wb') as f: 
            for chunk in r.iter_content(chunk_size = 1024*1024): 
                if chunk: 
                    f.write(chunk) 
        print("%s downloaded!\n"%file_name)
    print("All reports downloaded!")
    return
def delete_index_reports(file_names):
    for file in file_names:     
        print("Deleting file:%s"%file) 
        os.remove(file)
        print("%s deleted!\n"%file)
    print("All reports deleted!")
    return
def scrape_vti_index_reports(file_names):
    vtiDF = pd.DataFrame(columns=['File','MC'])
    for x in range(len(file_names)):
        try:
            pdf = PdfFileReader(file_names[x])
            pdf_page = pdf.getPage(1)
            pdf_text = pdf_page.extractText()
            mc = re.findall('INDEX MARKET CAP\d+,\d+,\d+', pdf_text, re.IGNORECASE)
            mc = mc[0].replace('INDEX MARKET CAP','')
            num = int(mc.replace(',',''))
            row = [file_names[x],num]
            vtiDF.loc[x] = row
        except:
            try:
                pdf = PdfFileReader(file_names[x])
                pdf_page = pdf.getPage(1)
                pdf_text = pdf_page.extractText()
                pdf_text = pdf_text.replace('\n','')
                mc = re.findall('Index Market Cap\d+,\d+,\d+', pdf_text)
                mc = mc[0].replace('Index Market Cap','')
                num = int(mc.replace(',',''))
                row = [file_names[x],num]
                vtiDF.loc[x] = row
            except:
                num = 'n/a'
                row = [file_names[x],num]
                vtiDF.loc[x] = row
    return vtiDF

In [4]:
vtiURL = "http://www.crsp.org/fact-sheet-archive"

In [5]:
reportURLS = get_report_links(vtiURL)
file_names = get_file_names(reportURLS)

In [6]:
reportURLS

['http://www.crsp.org/files/crsptm1_quarterly_report-december2019.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-september2019.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-june2019.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-march2019.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-december2018.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-september2018.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-june2018.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-march2018.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-december2017.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-september2017.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-june2017.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-march2017.pdf',
 'http://www.crsp.org/files/crsptm1_quarterly_report-dec2016.pdf',
 'http://www.crsp.org/files/CRSPTM1%20Quarterly%20Report-Sept2016.pdf',
 'http://www.cr

In [7]:
download_index_reports(reportURLS)

crsptm1_quarterly_report-december2019.pdf downloaded!

crsptm1_quarterly_report-september2019.pdf downloaded!

crsptm1_quarterly_report-june2019.pdf downloaded!

crsptm1_quarterly_report-march2019.pdf downloaded!

crsptm1_quarterly_report-december2018.pdf downloaded!

crsptm1_quarterly_report-september2018.pdf downloaded!

crsptm1_quarterly_report-june2018.pdf downloaded!

crsptm1_quarterly_report-march2018.pdf downloaded!

crsptm1_quarterly_report-december2017.pdf downloaded!

crsptm1_quarterly_report-september2017.pdf downloaded!

crsptm1_quarterly_report-june2017.pdf downloaded!

crsptm1_quarterly_report-march2017.pdf downloaded!

crsptm1_quarterly_report-dec2016.pdf downloaded!

CRSPTM1%20Quarterly%20Report-Sept2016.pdf downloaded!

CRSPTM1%20Quarterly%20Report-June2016.pdf downloaded!

CRSPTM1%20Quarterly%20Report-March2016.pdf downloaded!

CRSPTM1%20Quarterly%20Report-Dec2015.pdf downloaded!

CRSPTM1%20Quarterly%20Report-Sept2015.pdf downloaded!

CRSPTM1%20Quarterly%20Report-June

In [8]:
scrape_vti_index_reports(file_names)

,File,MC
0,crsptm1_quarterly_report-december2019.pdf,31780092
1,crsptm1_quarterly_report-september2019.pdf,29401430
2,crsptm1_quarterly_report-june2019.pdf,29290038
3,crsptm1_quarterly_report-march2019.pdf,28331596
4,crsptm1_quarterly_report-december2018.pdf,25089028
5,crsptm1_quarterly_report-september2018.pdf,29653118
6,crsptm1_quarterly_report-june2018.pdf,27900058
7,crsptm1_quarterly_report-march2018.pdf,27139040
8,crsptm1_quarterly_report-december2017.pdf,27452976
9,crsptm1_quarterly_report-september2017.pdf,25999436


In [9]:
delete_index_reports(file_names)

Deleting file:crsptm1_quarterly_report-december2019.pdf
crsptm1_quarterly_report-december2019.pdf deleted!

Deleting file:crsptm1_quarterly_report-september2019.pdf
crsptm1_quarterly_report-september2019.pdf deleted!

Deleting file:crsptm1_quarterly_report-june2019.pdf
crsptm1_quarterly_report-june2019.pdf deleted!

Deleting file:crsptm1_quarterly_report-march2019.pdf
crsptm1_quarterly_report-march2019.pdf deleted!

Deleting file:crsptm1_quarterly_report-december2018.pdf
crsptm1_quarterly_report-december2018.pdf deleted!

Deleting file:crsptm1_quarterly_report-september2018.pdf
crsptm1_quarterly_report-september2018.pdf deleted!

Deleting file:crsptm1_quarterly_report-june2018.pdf
crsptm1_quarterly_report-june2018.pdf deleted!

Deleting file:crsptm1_quarterly_report-march2018.pdf
crsptm1_quarterly_report-march2018.pdf deleted!

Deleting file:crsptm1_quarterly_report-december2017.pdf
crsptm1_quarterly_report-december2017.pdf deleted!

Deleting file:crsptm1_quarterly_report-september2017